
# Workshop Notebook 1: LLM Fundamentals & Building a Multimodal Chatbot

Welcome to the first part of our workshop! The goal of this notebook is to understand and practice the fundamental building blocks of Large Language Model (LLM) applications. We'll start simple and build our way up to a fun, interactive AI application: **The Creative Travel Planner** with optional **image** generation and **audio** responses.

---

### What you'll learn
1. Core LLM concepts (tokens, parameters, prompts)  
2. Making your **first Gemini API call** with `generate_content`  
3. Tuning **parameters** (`temperature`, `top_p`, `max_output_tokens`)  
4. Designing **system prompts / personas**  
5. **Streaming** responses (token‑by‑token)  
6. Adding simple **conversation memory**  
7. Building a **Travel Planner** (text → optional image → audio)

We use **Gemini 2.0 Flash**, a fast, multimodal model.

---

> **Docs & links:**
> - Gemini Quickstart → https://ai.google.dev/gemini-api/docs/quickstart  
> - Text generation → https://ai.google.dev/gemini-api/docs/text-generation  
> - Image generation → https://ai.google.dev/gemini-api/docs/image-generation  
> - Safety guidance → https://ai.google.dev/gemini-api/docs/safety-guidance   
> - Gradio Chat guide → https://www.gradio.app/guides/creating-a-chatbot-fast


## 0) Setup & Install
First, we need to install the necessary Python libraries.



In [ ]:
# Install required libraries
!pip -q install google-genai gradio gTTS pillow




## 1) API Key Configuration

To use the Gemini API, you need an API key.

1. Go to [Google AI Studio](https://aistudio.google.com/) and create an API key.
2. In Colab, click the Key icon (Secrets) in the left sidebar.
3. Create a new secret named `GEMINI_API_KEY` and paste your key.
4. Or set an environment variable in a cell: `os.environ['GEMINI_API_KEY'] = 'YOUR_KEY'`.

In [ ]:
#import libraries
import os, getpass
from google import genai

GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY") or getpass.getpass("🔑 Enter your GEMINI_API_KEY: ")
#genai.configure(api_key=GEMINI_API_KEY)

MODEL_ID = "gemini-2.0-flash"  # fast, multimodal
print("Gemini configured!")



## 2) LLM Basics (Quick Overview)

Before we start building, let’s understand a few key concepts that influence how LLMs generate text.

In this notebook we are using Gemini 2.0 Flash, part of Google’s Gemini family of generative AI models.


For more technical details, see Google’s official documentation:
 [Gemini 2.0 Flash – Model Overview](https://cloud.google.com/vertex-ai/generative-ai/docs/models/gemini/2-0-flash)

| Parameter | Description | Typical Range / Default |
|------------|--------------|--------------------------|
| **temperature** | Controls creativity/randomness in responses. Lower = more focused; higher = more diverse. | `0.0 – 2.0` (default ≈ `1.0`) |
| **top_p** | Considers only tokens within cumulative probability *p*. | `0.1 – 1.0` (default ≈ `0.95`) |
| **top_k** | Considers only the top *k* most likely tokens. | default: `64 (fixed)`|
| **max_output_tokens** | Maximum number of tokens to generate. | default = `8,192`|
| **seed** | (Optional) Makes responses reproducible. | Integer value |

> **Tip:** Increase `temperature` for creativity; lower it for accuracy.  
> Use `top_p` or `top_k` to balance diversity and focus.

**Reference:** [Adjusting Parameter Values — Vertex AI Docs](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/adjust-parameter-values)



## 3) Your first Gemini call

In [ ]:
# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client(api_key=GEMINI_API_KEY)

response = client.models.generate_content(
    model=MODEL_ID,
    contents="Hello Gemini! What can you do?"
)
print(response.text)

In [ ]:
#Display Markdown
from IPython.display import Markdown, display

display(Markdown(response.text))


## 4) Demo #1 — Minimal chatbot (text only)


In [ ]:
def chat_minimal(user_text):
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=user_text
    )
    return response.text


In [ ]:
import gradio as gr

with gr.Blocks(theme="soft") as demo1:
    gr.Markdown("### Minimal Chat (Gemini API Client)")
    inp = gr.Textbox(label="Ask anything")
    out = gr.Textbox(label="Response", lines=6)
    gr.Button("Send").click(chat_minimal, inp, out)




In [ ]:
demo1.launch()

In [ ]:
demo1.close()


## 5) Parameters Playground


In [ ]:
def generate_with_params(prompt, temperature, top_p, max_tokens):
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            "temperature": float(temperature),
            "top_p": float(top_p),
            "max_output_tokens": int(max_tokens),
        },
    )
    return response.text.strip()

In [ ]:
# More creative and free-flowing
print(generate_with_params("Generate a marketing slogan for a new coffee shop in Dublin.", temperature=2, top_p=1, max_tokens=500))

# Balanced, informative
print("\n-----------------------------------")
print(generate_with_params("Generate a marketing slogan for a new coffee shop in Dublin..", temperature=0.7, top_p=1, max_tokens=200))

# Precise and conservative
print("\n-----------------------------------")
print(generate_with_params("Generate a marketing slogan for a new coffee shop in Dublin..", temperature=0.3, top_p=1, max_tokens=100))


In [ ]:
with gr.Blocks(theme="soft") as demo2:
    gr.Markdown("### 🎛️ Parameters Playground")

    prompt = gr.Textbox(
        value="Give me 3 creative day-trip ideas near Lisbon.",
        lines=2,
        label="Prompt"
    )

    with gr.Row():
        temperature = gr.Slider(0.0, 2.0, 1.2, step=0.1, label="temperature")
        top_p = gr.Slider(0.1, 1.0, 0.9, step=0.05, label="top_p")
        max_tokens = gr.Slider(64, 512, 300, step=8, label="max_output_tokens")

    out = gr.Textbox(lines=10, label="Response")

    gr.Button("Generate").click(
        generate_with_params,
        inputs=[prompt, temperature, top_p, max_tokens],
        outputs=out
    )

#demo2.launch(share=True, inline=False)


In [ ]:
demo2.launch()

In [ ]:
demo2.close()



## 6) System prompts (personas)


In this section, we'll explore how **system prompts** (or **personas**) influence how a language model responds.  
A *system prompt* sets the *behavior, tone, and role* of the model — essentially telling it **how** to respond, not just **what** to respond to.

####  How Chat Models Process Messages
Most modern LLM APIs (like OpenAI and Gemini) use a **multi-message format** that mirrors a conversation.  
Each message has a *role* that helps the model interpret context correctly:

| Role | Description | Example |
|------|--------------|----------|
| **system** | Sets the model’s overall behavior or persona | “You are a friendly tutor who explains concepts clearly.” |
| **user** | Contains the user’s prompt or question | “Explain why the sky is blue.” |
| **assistant** | Holds the model’s previous response (for ongoing chats) | “The sky appears blue because of Rayleigh scattering…” |

> In our case, we manually include the **system** and **user** roles inside a single text prompt.  
> This is enough for simple, single-turn chats, but for multi-turn conversation, the Gemini API also supports structured message objects.

---

#### Prompt Inspiration & Examples

If you’d like to explore examples of well-designed prompts, check out:
- [Google Cloud Vertex AI Prompt Gallery](https://cloud.google.com/vertex-ai/generative-ai/docs/prompt-gallery)  
- [Gemini Prompt Design Guide](https://ai.google.dev/gemini-api/docs/prompting)  
- [OpenAI Prompt Examples](https://platform.openai.com/docs/examples)  

Further Reading - [System Instructions - Vertex AI docs](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/system-instructions)

---

Next, we’ll build an interactive **Persona Playground** in Gradio that lets you select a system prompt (persona), adjust the model’s creativity, and observe how the tone of the AI’s response changes in real time.


In [ ]:
# Define a few example personas
PERSONAS = {
    "Friendly Tutor": "You are a friendly tutor who explains concepts simply with small examples.",
    "Concise Analyst": "You are a precise analyst who answers briefly with bullet points.",
    "Creative Storyteller": "You are a playful storyteller who responds vividly and imaginatively.",
}

# Persona-based chat function
def persona_chat(user_text, persona, temperature):
    sys_prompt = PERSONAS[persona]
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"System: {sys_prompt}\nUser: {user_text}\nAI:",
        config={
            "temperature": float(temperature)
        }
    )
    return response.text.strip()

In [ ]:
from IPython.display import Markdown, display

response = persona_chat("Explain why the sky looks blue.", "Friendly Tutor", 0.7)
display(Markdown(response))

In [ ]:
from IPython.display import Markdown, display

response = persona_chat("Explain why the sky looks blue.", "Creative Storyteller", 2.0)
display(Markdown(response))

In [ ]:
import gradio as gr

with gr.Blocks(theme="soft") as demo3:
    gr.Markdown("### 🎭 Persona Playground")

    user_text = gr.Textbox(
        placeholder="Ask a question or give a prompt...",
        label="Your message",
        lines=2
    )

    with gr.Row():
        persona = gr.Dropdown(
            choices=list(PERSONAS.keys()),
            value="Friendly Tutor",
            label="Persona"
        )
        temperature = gr.Slider(
            minimum=0.0,
            maximum=2.0,
            value=1.0,
            step=0.1,
            label="Temperature"
        )

    out = gr.Markdown(label="Response")  # renders Markdown output

    gr.Button("Generate").click(
        fn=persona_chat,
        inputs=[user_text, persona, temperature],
        outputs=out
    )

#demo3.launch(share=True, inline=False)


In [ ]:
demo3.launch()

In [ ]:
demo3.close()


## 7) Streaming responses


You can choose whether the model generates streaming responses or non-streaming responses. For streaming responses, you receive each response as soon as its output token is generated. For non-streaming responses, you receive all responses after all of the output tokens are generated.

In [ ]:
def stream_to_stdout(prompt, temperature=0.7):
    """
    Streams Gemini responses token-by-token in real time

    """
    out = []
    print("Streaming response:\n")

    stream = client.models.generate_content_stream(
        model="gemini-2.0-flash",
        contents=prompt,
        config={"temperature": float(temperature)},
    )

    for event in stream:
        if event.text:
            print(event.text, end="", flush=True)
            out.append(event.text)

    print()  # newline for formatting
    return "".join(out)

# Test it
_ = stream_to_stdout("Give me 5 carry-on packing tips for a 2-day work trip.")



## 8) Simple memory chat


LLMs like Gemini are **stateless by default** — meaning they don’t remember anything from previous messages.  
Each API call is processed independently, so if you ask a follow-up question without repeating the context,  
the model won’t know what you’re referring to.

To make a chatbot feel *conversational*, we can simulate memory by:
1. Keeping a **history** of user and AI messages.
2. Sending that history back to the model as part of the prompt each time.

In this section, we’ll add lightweight “short-term memory” using a `history` list.

**Reference:**  
- [Gemini API: Text Generation](https://ai.google.dev/gemini-api/docs/text-generation)  



In [ ]:
# First call: introduce yourself
resp1 = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Hello! My name is ------."
)
print("Response 1:", resp1.text)

# Second call: ask a follow-up without context
resp2 = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="What's my name?"
)
print("\nResponse 2:", resp2.text)


In [ ]:
def chat_with_memory(user_text, history,
                     system_prompt="You are a concise, friendly assistant.",
                     max_turns=6, temperature=0.7):
    """
    Builds lightweight conversational memory by replaying
    recent turns as context for the model.
    """
    # Keep only the last few turns to stay within token limits
    turns = history[-max_turns:] if history else []

    # Construct conversation transcript
    transcript = [f"System: {system_prompt}"]
    for user, ai in turns:
        transcript += [f"User: {user}", f"AI: {ai}"]
    transcript += [f"User: {user_text}", "AI:"]

    # Generate a response
    resp = client.models.generate_content(
        model="gemini-2.0-flash",
        contents="\n".join(transcript),
        config={"temperature": float(temperature)},
    )
    return resp.text.strip()


In [ ]:
SYSTEM_PROMPT = "You are a friendly assistant who remembers recent details."

with gr.Blocks(theme="soft") as demo_memory:
    gr.Markdown("### Memory Chat (Lightweight Demo)")

    chatbox = gr.Chatbot(height=320)
    msg = gr.Textbox(label="Say something", placeholder="e.g., My name is ----. Remember it.")
    temperature = gr.Slider(0.0, 1.0, 0.7, step=0.05, label="Temperature")
    state = gr.State([])

    def on_submit(user_text, temp, st):
        st = st or []
        ai = chat_with_memory(user_text, st, SYSTEM_PROMPT, temperature=temp)
        st.append((user_text, ai))
        return st, st

    msg.submit(on_submit, [msg, temperature, state], [chatbox, state])

# Uncomment to run:
# demo_memory.launch(share=True, inline=False)


In [ ]:
demo_memory.launch()

In [ ]:
demo_memory.close()

---

## Business Applications of What You’ve Learned

So far, you’ve explored **how LLMs communicate** — from text generation to parameter tuning, system prompts and memory.

These skills are not just technical curiosities — they’re the foundation for **real business impact** across industries.  

### Real-World Use Cases
| Domain | Example Application | Description |
|---------|---------------------|--------------|
| **Travel & Hospitality** | AI Travel Planner | Personalized trip recommendations, itinerary generation, and visual previews for destinations. |
| **Retail & E-Commerce** | AI Product Assistant | Product description generation, comparison tools, and conversational shopping assistants. |
| **Customer Support** | Chatbots with Memory | Context-aware responses that simulate human-like understanding of user history. |
| **Marketing & Communications** | Content Generation | Email personalization, social media copywriting, and campaign ideation. |
| **Education & Training** | AI Tutors | Personalized learning assistants that explain, quiz, and summarize material dynamically. |

> Generating and refining content in this way is one of the most **common and high-impact LLM use cases** today.  

---

## Coming Next: Retrieval-Augmented Generation (RAG)

In the next phase, we’ll bridge what you’ve learned with **data grounding** — teaching your chatbot to *access knowledge*
beyond what it was trained on.

You’ll learn how to:
- Connect LLMs to your **own data** (PDFs, websites, company documents)  
- Retrieve and summarize relevant context dynamically  
- Reduce hallucinations by grounding responses in facts  
- Build a **domain-specific chatbot** that truly knows your business  

This marks the shift from **creative generation** to **knowledge-powered intelligence** —
the core of most enterprise AI systems today.

---

📚 **Further Reading**
- [Google Cloud: 1,001 real-world gen AI use cases from the world's leading organizations](https://cloud.google.com/transform/101-real-world-generative-ai-use-cases-from-industry-leaders)

---

### 🌟 Reflection

> Think of a process in your own field or company that involves **text, images, or decisions**.  
> Could an AI assistant like the one you built today help streamline it?  
> If so, you’ve just identified your first **AI use case!**. 👏

---


## 9) Bonus Section - Creative Travel Planner

Now, let's combine everything we've learned—API calls, system prompts, and multimodal capabilities—to build our final application.

The **Creative Travel Planner** will:
1.  Take a user's travel request.
2.  Generate a text-based itinerary or set of ideas using a Gemini model.
3.  Optionally, generate a beautiful image of the destination using Stable Diffussion.
4.  Optionally, convert the text response into speech using Google's Text-to-Speech (gTTS) model.

### **Image Generation Setup: Hugging Face API Token**

To use a free and simple image generation model, we'll use Stable Diffusion through the Hugging Face API. This requires a separate, free API token.

**Step-by-Step Guide to Get Your HF Token:**
1.  **Go to Hugging Face:**
    * Open your browser and navigate to [https://huggingface.co/](https://huggingface.co/).
2.  **Create an Account:**
    * Sign up for a free account.
3.  **Find Your Access Tokens:**
    * Click on your profile picture in the top-right corner, then go to **Settings**.
    * In the left-hand menu, click on **Access Tokens**.
4.  **Create a New Token:**
    * Click the "**New token**" button. Give it a name (e.g., "WAI-Workshop") and set the role to "**read**". Click "**Generate a token**".
5.  **Copy and Add to Colab Secrets:**
    * Copy the generated token.
    * In your Colab notebook, click the **Key icon (Secrets)** in the left sidebar.
    * Create a new secret named `HF_TOKEN` and paste your token as the value. Make sure "Notebook access" is enabled.

In [ ]:
from PIL import Image
import requests
import io
import uuid
from gtts import gTTS
from google.colab import userdata

In [ ]:
# --- Load the Hugging Face Token ---
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
except userdata.SecretNotFoundError:
    print('Secret "HF_TOKEN" not found. Please follow the setup instructions.')
    HF_TOKEN = None

In [ ]:
# --- Model Name Constants ---
TEXT_MODEL_NAME = "gemini-2.0-flash"
IMAGE_MODEL_ID = "stabilityai/stable-diffusion-xl-base-1.0"

def generate_destination_image(prompt: str):
    """Generates an image using the Hugging Face Inference API for Stable Diffusion XL."""
    if not HF_TOKEN:
        print(" Image generation failed: Hugging Face token not found.")
        return None
    print(f"Generating image with {IMAGE_MODEL_ID}...")
    api_url = f"https://api-inference.huggingface.co/models/{IMAGE_MODEL_ID}"
    headers = {"Authorization": f"Bearer {HF_TOKEN}"}
    payload = {"inputs": f"A beautiful, vibrant, photorealistic travel advertisement photo of {prompt}, cinematic lighting, 8k"}
    try:
        response = requests.post(api_url, headers=headers, json=payload, timeout=90)
        response.raise_for_status()
        image_bytes = response.content
        return Image.open(io.BytesIO(image_bytes))
    except requests.exceptions.RequestException as e:
        print(f" Image generation failed: {e}")
        if response.status_code == 503:
            print("The model might be loading on the Hugging Face servers. Please try again shortly.")
        else:
            print(f"Full response: {response.text}")
        return None


In [ ]:
def synthesize_audio(text: str, directory="/tmp"):
    """Converts text to speech using gTTS and saves it as an MP3 file."""
    print(" Synthesizing audio...")
    os.makedirs(directory, exist_ok=True)
    path = os.path.join(directory, f"tts_{uuid.uuid4().hex[:8]}.mp3")
    try:
        tts = gTTS(text)
        tts.save(path)
        return path
    except Exception as e:
        print(f" Audio synthesis failed: {e}")
        return None

In [ ]:
#
TRAVEL_SYSTEM_PROMPT = """You are a creative and enthusiastic travel planner.
Your goal is to provide exciting and practical travel ideas.
Be concise and use bullet points for itineraries.
IMPORTANT: Do not use any markdown formatting. Do not use asterisks, hashes, or any other special characters. Respond in plain text only.
"""

def plan_my_trip(user_request, generate_image_flag, generate_audio_flag):
    """Main function to handle Gradio inputs and generate multimodal outputs."""
    print(f" Processing request: '{user_request}'")

    # --- 1. Generate Text Response ---
    full_prompt = f"System: {TRAVEL_SYSTEM_PROMPT}\nUser: {user_request}\nAI:"
    response = client.models.generate_content(
        model=TEXT_MODEL_NAME,
        contents=full_prompt
    )
    # This single, clean text variable is used for both display and audio
    plain_text_response = response.text.strip()

    # --- 2. Generate Image (if requested) ---
    image_output = None
    if generate_image_flag:
        image_output = generate_destination_image(user_request)

    # --- 3. Generate Audio (if requested) ---
    audio_output_path = None
    if generate_audio_flag:
        # We can now directly use the clean text from the model
        audio_output_path = synthesize_audio(plain_text_response)

    return plain_text_response, image_output, audio_output_path

In [ ]:
import gradio as gr

with gr.Blocks(theme=gr.themes.Soft()) as demo_travel:
    gr.Markdown("# 🌍 Creative Travel Planner")
    gr.Markdown("Describe your dream trip, and let the AI assistant plan it for you!")

    with gr.Row():
        with gr.Column(scale=3):
            user_input = gr.Textbox(
                label="Your Trip Request",
                placeholder="e.g., Plan a 3-day romantic weekend in Rome",
                lines=2
            )
            with gr.Row():
                want_image = gr.Checkbox(label="🖼️ Generate an Image", value=True)
                want_audio = gr.Checkbox(label="🗣️ Generate Audio", value=False)

            submit_button = gr.Button("Plan my Trip", variant="primary")

        with gr.Column(scale=2):
            plan_output_text = gr.Textbox(label="Itinerary & Suggestions", lines=10)
            plan_output_image = gr.Image(type="pil", label="Destination Visual")
            plan_output_audio = gr.Audio(label="Spoken Itinerary", autoplay=False)

    gr.Examples(
        examples=[
            ["A 2-day budget-friendly trip to Lisbon", True, False],
            ["Suggest a relaxing beach holiday in Thailand", True, True],
            ["What are the best things to do on a family trip to Dublin?", False, False],
        ],
        inputs=[user_input, want_image, want_audio]
    )

    submit_button.click(
        fn=plan_my_trip,
        inputs=[user_input, want_image, want_audio],
        outputs=[plan_output_text, plan_output_image, plan_output_audio]
    )

print("Gradio app ready. Launching...")
demo_travel.launch(debug=True)

In [ ]:
demo_travel.close()